In [1]:
# Hack to import from a parent directory
import sys
path = '..'
if path not in sys.path:
    sys.path.append(path)

In [2]:
#PyData imports
import numpy as np
import pandas as pd
import matplotlib as plt

import stan
import arviz
import os
import nest_asyncio
nest_asyncio.apply()

In [3]:
#Data to be passed to Stan.
state_dim = 4
temp_ref = 283
temp_rise = 5 #High estimate of 5 celsius temperature rise by 2100.
prior_scale_factor = 0.25
obs_error_scale = 0.1
obs_every = 10 #Observations every 10 hours.
t = 3000 #Total time span of ODE simulation.
x_hat0 = [99.69113179, 1.94470102, 1.87967951, 2.01791824] #Originally sampled values used for Euler-Maruyama solution.
y_full = pd.read_csv(os.path.join('generated_data/', 'SAWB-ECA-SS_no_CO2_trunc_short_2021_12_23_22_21_sample_y_t_3000_dt_0-01_sd_scale_0-25.csv'))
y = y_full[y_full['hour'] <= t][1:]
ts = y['hour'].tolist()
N_t = len(ts)
#y = np.array(y.drop(columns = 'hour')).tolist() #Convert data observations to list of rows to correspond to Stan's array of vectors type.
y = np.array(y.drop(columns = 'hour')).T.tolist() #Convert data observations to list of columns to correspond to Stan's array of vectors type.

#Parameter prior distribution parameters in order of [mean, lower, upper]
u_Q_ref_prior_dist_params = [0.22, 1e-2, 1]
Q_prior_dist_params = [0.001, 0, 0.1]
a_MSA_prior_dist_params = [0.5, 0, 1]
K_DE_prior_dist_params = [1000, 100, 5000]
K_UE_prior_dist_params = [0.1, 1e-2, 1]
V_DE_ref_prior_dist_params = [0.04, 1e-3, 1]
V_UE_ref_prior_dist_params = [0.005, 1e-4, 0.1]
Ea_V_DE_prior_dist_params = [40, 5, 80]
Ea_V_UE_prior_dist_params = [30, 5, 80]
r_M_prior_dist_params = [0.00016667, 1e-5, 0.1]
r_E_prior_dist_params = [0.0002, 1e-5, 0.1]
r_L_prior_dist_params = [0.0004, 1e-5, 0.1]

In [4]:
data_dict = {
    'state_dim': state_dim,
    'N_t': N_t,
    'ts': ts,
    'y': y,
    'temp_ref': temp_ref,
    'temp_rise': temp_rise,
    'prior_scale_factor': prior_scale_factor,
    'obs_error_scale': obs_error_scale,
    'x_hat0': x_hat0,
    'u_Q_ref_prior_dist_params': u_Q_ref_prior_dist_params,
    'Q_prior_dist_params': Q_prior_dist_params,
    'a_MSA_prior_dist_params': a_MSA_prior_dist_params,
    'K_DE_prior_dist_params': K_DE_prior_dist_params,
    'K_UE_prior_dist_params': K_UE_prior_dist_params,
    'V_DE_ref_prior_dist_params': V_DE_ref_prior_dist_params,
    'V_UE_ref_prior_dist_params': V_UE_ref_prior_dist_params,
    'Ea_V_DE_prior_dist_params': Ea_V_DE_prior_dist_params,
    'Ea_V_UE_prior_dist_params': Ea_V_UE_prior_dist_params,
    'r_M_prior_dist_params': r_M_prior_dist_params,
    'r_E_prior_dist_params': r_E_prior_dist_params,
    'r_L_prior_dist_params': r_L_prior_dist_params
}

In [5]:
AWB_ECA_stan_file = open('AWB-ECA_SS_no_CO2_cont_time.stan').read()
print(AWB_ECA_stan_file)

functions {

  // Temperature function for ODE forcing.
  real temp_func(real t, real temp_ref, real temp_rise) {
    return temp_ref + (temp_rise * t) / (80 * 24 * 365) + 10 * sin((2 * pi() / 24) * t) + 10 * sin((2 * pi() / (24 * 365)) * t);
  }

  // Exogenous SOC input function.
  real i_s_func(real t) {
    return 0.001 + 0.0005 * sin((2 * pi() / (24 * 365)) * t);
  }

  // Exogenous DOC input function.
  real i_d_func(real t) {
    return 0.0001 + 0.00005 * sin((2 * pi() / (24 * 365)) * t);
  }

  // Function for enforcing Arrhenius temperature dependency of ODE parameter.
  real arrhenius_temp(real input, real temp, real Ea, real temp_ref) {
    return input * exp(-Ea / 0.008314 * (1 / temp - 1 / temp_ref));
  }

  // Function for enforcing linear temperature dependency of ODE parameter.
  real linear_temp(real input, real temp, real Q, real temp_ref) {
    return input - Q * (temp - temp_ref);
  }

  /*
  AWB-ECA (equilibrium chemistry approximation) variant of AWB model.
  C[1]

In [6]:
AWB_ECA_stan_model = stan.build(program_code = AWB_ECA_stan_file, data = data_dict, random_seed = 123)

Building...



Building: 44.5s, done.Messages from stanc:
Warning in '/var/folders/mr/47dcvttx6_5dqt_p2ktdplc80000gn/T/httpstan_bs7_njbj/model_34lbjcpq.stan', line 91, column 16: Use of normal_cdf without a vertical bar (|) between the first two arguments is deprecated.
Warning in '/var/folders/mr/47dcvttx6_5dqt_p2ktdplc80000gn/T/httpstan_bs7_njbj/model_34lbjcpq.stan', line 92, column 16: Use of normal_cdf without a vertical bar (|) between the first two arguments is deprecated.
Warning in '/var/folders/mr/47dcvttx6_5dqt_p2ktdplc80000gn/T/httpstan_bs7_njbj/model_34lbjcpq.stan', line 177, column 35: The variable AWB_ECA_ODE may not have been assigned a value before its use.
Warning in '/var/folders/mr/47dcvttx6_5dqt_p2ktdplc80000gn/T/httpstan_bs7_njbj/model_34lbjcpq.stan', line 165, column 116: A control flow statement depends on parameter(s): r_L.
Warning in '/var/folders/mr/47dcvttx6_5dqt_p2ktdplc80000gn/T/httpstan_bs7_njbj/model_34lbjcpq.stan', line 165, column 90: A control flow statement depends

In [ ]:
AWB_ECA_fit = AWB_ECA_stan_model.sample(num_chains = 3, num_samples = 1000)